In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,26877
2,Huelva,Confirmados PDIA 14 días,1292
3,Huelva,Tasa PDIA 14 días,"251,76841982189137"
4,Huelva,Confirmados PDIA 7 días,631
5,Huelva,Total Confirmados,27051
6,Huelva,Curados,22418
7,Huelva,Fallecidos,357


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  26877.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6991.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1229 personas en los últimos 7 días 

Un positivo PCR cada 529 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,26877.0,631.0,1292.0,513170.0,122.961202,251.768420,159.0
Condado-Campiña,8118.0,338.0,684.0,156231.0,216.346308,437.813238,80.0
Huelva-Costa,15937.0,239.0,511.0,289548.0,82.542445,176.481965,69.0
Huelva (capital),6991.0,117.0,272.0,143837.0,81.342075,189.102943,31.0
Almonte,1233.0,109.0,214.0,24507.0,444.770882,873.219896,27.0
Cartaya,1147.0,74.0,137.0,20083.0,368.470846,682.168999,19.0
Palos de la Frontera,541.0,32.0,59.0,11742.0,272.525975,502.469767,14.0
Moguer,980.0,41.0,89.0,21867.0,187.497142,407.005991,11.0
Bollullos Par del Condado,723.0,20.0,51.0,14387.0,139.014388,354.486689,10.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
San Bartolomé de la Torre,274.0,19.0,43.0,3761.0,505.184791,1143.312949,6.0
Almonte,1233.0,109.0,214.0,24507.0,444.770882,873.219896,27.0
Rociana del Condado,435.0,43.0,65.0,7939.0,541.629928,818.742915,3.0
Sanlúcar de Guadiana,6.0,3.0,3.0,403.0,744.416873,744.416873,0.0
Cartaya,1147.0,74.0,137.0,20083.0,368.470846,682.168999,19.0
Lucena del Puerto,183.0,14.0,20.0,3261.0,429.316161,613.308801,2.0
Bonares,219.0,15.0,32.0,6060.0,247.524752,528.052805,1.0
Palos de la Frontera,541.0,32.0,59.0,11742.0,272.525975,502.469767,14.0
Condado-Campiña,8118.0,338.0,684.0,156231.0,216.346308,437.813238,80.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Ayamonte,1254.0,1.0,5.0,21104.0,4.738438,23.692191,2.0,0.200000
Villalba del Alcor,470.0,2.0,8.0,3366.0,59.417706,237.670826,0.0,0.250000
Gibraleón,556.0,3.0,10.0,12737.0,23.553427,78.511423,0.0,0.300000
Villanueva de los Castillejos,157.0,1.0,3.0,2825.0,35.398230,106.194690,1.0,0.333333
Higuera de la Sierra,32.0,1.0,3.0,1291.0,77.459334,232.378002,0.0,0.333333
Cortegana,192.0,1.0,3.0,4602.0,21.729683,65.189048,0.0,0.333333
Trigueros,248.0,7.0,19.0,7862.0,89.035869,241.668787,3.0,0.368421
Lepe,2043.0,8.0,21.0,27880.0,28.694405,75.322812,7.0,0.380952
Bollullos Par del Condado,723.0,20.0,51.0,14387.0,139.014388,354.486689,10.0,0.392157
